In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 41.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=524447f27ade77f696800319a777e17a0d8d8713c65ce20fc3a915e86f4976bf
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
import datetime
import time
import random
spark = SparkSession.builder.master('local[*]').getOrCreate()


Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:

In [ ]:
schema = T.StructType([
                       T.StructField("id", T.IntegerType(), True),
                T.StructField("timestamp", T.LongType(), True),
                T.StructField("type", T.StringType(), True),
                T.StructField("page_id", T.IntegerType(), True),
                T.StructField("tag", T.StringType(), True),
                T.StructField("sign", T.BooleanType(), True)])



In [ ]:
data =[(12345, 1667627426, "click", 101, "sport", True),
       (12345, 1380412800, "move", 102, "sport", True),
    (12346, 1211760000, "visit", 103, "IT", True),
    (12344, 1522713600, "scroll", 104, "education", True),
    (12344, 1423612800, "click", 105, "education", False),
    (12347, 1337644800, "move", 106, "medicine", True),
   (12346, 1026000000, "click", 107, "IT", False),
   (12346, 1332460800, "click", 108, "IT", False),
   (12345, 1276473600, "visit", 109, "sport", True),
    (12347, 1289347200, "click", 110, "medicine", True),
    (12343, 985910400, "move", 111, "sport", True),
    (12348, 1542153600, "scroll", 112, "politics", False),
    (12348, 1557100800, "click", 113, "politics", False),
    (12345, 1547424000, "move", 114, "sport", True),
   (12345, 1438732800, "scroll", 115, "sport", True)]


Создайте датафрейм с описанной выше схемой данных

In [ ]:
df = spark.createDataFrame(data=data,schema=schema)
df = df.select(*[i for i in df.columns if i != 'timestamp'],
          F.from_unixtime('timestamp').alias('date_time'))
df.show()

   

+-----+------+-------+---------+-----+-------------------+
|   id|  type|page_id|      tag| sign|          date_time|
+-----+------+-------+---------+-----+-------------------+
|12345| click|    101|    sport| true|2022-11-05 05:50:26|
|12345|  move|    102|    sport| true|2013-09-29 00:00:00|
|12346| visit|    103|       IT| true|2008-05-26 00:00:00|
|12344|scroll|    104|education| true|2018-04-03 00:00:00|
|12344| click|    105|education|false|2015-02-11 00:00:00|
|12347|  move|    106| medicine| true|2012-05-22 00:00:00|
|12346| click|    107|       IT|false|2002-07-07 00:00:00|
|12346| click|    108|       IT|false|2012-03-23 00:00:00|
|12345| visit|    109|    sport| true|2010-06-14 00:00:00|
|12347| click|    110| medicine| true|2010-11-10 00:00:00|
|12343|  move|    111|    sport| true|2001-03-30 00:00:00|
|12348|scroll|    112| politics|false|2018-11-14 00:00:00|
|12348| click|    113| politics|false|2019-05-06 00:00:00|
|12345|  move|    114|    sport| true|2019-01-14 00:00:0

Вывести топ-5 самых активных посетителей сайта

In [ ]:
df.groupBy("id").agg({"type":"count"}).orderBy("count(type)", ascending = False).show(5)

+-----+-----------+
|   id|count(type)|
+-----+-----------+
|12345|          5|
|12346|          3|
|12347|          2|
|12344|          2|
|12348|          2|
+-----+-----------+
only showing top 5 rows



Посчитать процент посетителей, у которых есть ЛК

In [ ]:
user_amount = df.count()
sign_amount = df.filter((F.col('sign')=='True')).count()
percent_LK = 100*sign_amount/user_amount
print("процент посетителей, у которых есть ЛК: " + str(percent_LK) + "%")

процент посетителей, у которых есть ЛК: 66.66666666666667%


Вывести топ-5 страниц сайта по показателю среднего кол-ва кликов на данной странице

In [ ]:
df.select('*').where(df["type"]=="click").groupBy("page_id").count().orderBy("count", ascending = False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    101|    1|
|    107|    1|
|    105|    1|
|    108|    1|
|    113|    1|
+-------+-----+
only showing top 5 rows



•   Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [ ]:
df_time = df.withColumn("time_range", F.floor(F.hour("date_time") / F.lit(4)))
df_time.show()

+-----+------+-------+---------+-----+-------------------+----------+
|   id|  type|page_id|      tag| sign|          date_time|time_range|
+-----+------+-------+---------+-----+-------------------+----------+
|12345| click|    101|    sport| true|2022-11-05 05:50:26|         1|
|12345|  move|    102|    sport| true|2013-09-29 00:00:00|         0|
|12346| visit|    103|       IT| true|2008-05-26 00:00:00|         0|
|12344|scroll|    104|education| true|2018-04-03 00:00:00|         0|
|12344| click|    105|education|false|2015-02-11 00:00:00|         0|
|12347|  move|    106| medicine| true|2012-05-22 00:00:00|         0|
|12346| click|    107|       IT|false|2002-07-07 00:00:00|         0|
|12346| click|    108|       IT|false|2012-03-23 00:00:00|         0|
|12345| visit|    109|    sport| true|2010-06-14 00:00:00|         0|
|12347| click|    110| medicine| true|2010-11-10 00:00:00|         0|
|12343|  move|    111|    sport| true|2001-03-30 00:00:00|         0|
|12348|scroll|    11

Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [ ]:
df_time.groupby("time_range")\
      .agg(F.count("*").alias("time_range_count"))\
      .orderBy("time_range_count", ascending = False)\
      .limit(1).show()

+----------+----------------+
|time_range|time_range_count|
+----------+----------------+
|         0|              14|
+----------+----------------+



Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов

In [ ]:
schema_2 = T.StructType([
                T.StructField("id", T.StringType(), True),
                T.StructField("user_id", T.IntegerType(), True),
                T.StructField("fio", T.StringType(), True),
               T.StructField("dateofcreate_LK", T.DateType(), True)])
from datetime import datetime
data_2 = [
    (1, 12343, "Сергеев Борис Джонсович", datetime(2018, 6, 5), ),
    (2, 12344, "Иванов Иван Сергеевич", datetime(2016, 1, 12),),
    (3, 12345, "Джеймс Джеймс Джеймcович", datetime(2011, 4, 25), ),
    (4, 12346, "Полякова Ирина Витальевна", datetime(2022, 11, 22), ),
    (5, 12348, "Михайлов Михаил Михайлович", datetime(2017, 3, 13), ),
    (6, 12349, "Маратов Марат Маратович", datetime(2017, 7, 7), ),
]

In [ ]:
df_users = spark.createDataFrame(data = data_2, schema = schema_2)
df_users.show()

+---+-------+--------------------+---------------+
| id|user_id|                 fio|dateofcreate_LK|
+---+-------+--------------------+---------------+
|  1|  12343|Сергеев Борис Джо...|     2018-06-05|
|  2|  12344|Иванов Иван Серге...|     2016-01-12|
|  3|  12345|Джеймс Джеймс Дже...|     2011-04-25|
|  4|  12346|Полякова Ирина Ви...|     2022-11-22|
|  5|  12348|Михайлов Михаил М...|     2017-03-13|
|  6|  12349|Маратов Марат Мар...|     2017-07-07|
+---+-------+--------------------+---------------+



Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт

In [ ]:
df_Stg = df_users.alias("lk").join(df.alias("web"),
                                    on = [F.col("lk.user_id") == F.col("web.id")],
                                    how = "left")

df_sport = df_Stg.select("fio", "tag")\
                      .filter((F.col('tag')=='Sport'))\
                      .select("fio")\
                      .dropDuplicates(["fio"])
df_sport.show(truncate=False)

+---+
|fio|
+---+
+---+

